In [1]:
import pandas as pd

Subir el fichero de navegacion

In [2]:
from google.colab import files
uploaded = files.upload()

Saving navegacion.csv to navegacion.csv


Crear el dataframe del archivo de navegacion.csv

In [ ]:
dfNavegacion = pd.read_csv("navegacion.csv",sep=";")
print(dfNavegacion.head(10))

Analisis Exploratorio de Datos (EDA)
  - Limpia de datos
  - Proceder a eliminar datos que no sean coherentes porque afectaria al resultado final

In [ ]:
# Comprobar cuantos gclid hay repetidos
print(dfNavegacion['gclid'].value_counts().head(50))

In [ ]:
# Comprobar si los gclid pertenecen al mismo usuario
print(dfNavegacion.groupby(['gclid','id_user'])['ts'].count()  )

In [ ]:
# Hemos visto que el gclid=0 pertenece a diferentes usuarios, lo elimino
# al considerar que no es un dato coherente
gclid0 = dfNavegacion[dfNavegacion['gclid']=='0'].index
print(gclid0)
dfNavegacion = dfNavegacion.drop(gclid0)

In [ ]:
# el gclid con 124 ocurrencias tambien lo elimino
# al considerar que no es un dato coherente
gclid124 = dfNavegacion[dfNavegacion['gclid']=='EAIaIQobChMIwPu5t4qs3AIVAQAAAB0BAAAAEAAYACAAEgJVzfD_BwE'].index
print(gclid124)
dfNavegacion = dfNavegacion.drop(gclid124)

Empezamos a procesar las URL:
  - Filtar por las url_landing no sean nulas
  - Separar los datos de la url en columnas

In [ ]:
# filtrar url no nulas
serieUrlLanding = dfNavegacion['url_landing'][dfNavegacion['url_landing'].notnull()]

# Superar en columnas la url utilizando split('&')
# Cada parte lo guardo en una nueva columna
print(dfNavegacion.columns)
dfNavegacion[['url_base','idUser','uuid_url','camp','adg','device','sl','adv','rec','vacia']] = serieUrlLanding.str.split(pat='&',n=10,expand=True)
print(dfNavegacion.columns)

In [ ]:
# Eliminamos columnas que estan repetidas (en cuanto al valor almacenado)
dfNavegacion = dfNavegacion.drop( ['idUser', 'uuid_url', 'rec', 'vacia'], axis=1 )
print(dfNavegacion.columns)

Obtener el modelo de coche

In [ ]:
#print(dfNavegacion['url_base'].head(50))
# url/modelo?parametro
# Para obtener el modelo de coche me quedo solo con lo que delante del interrogante
dfNavegacion['url_base'] = dfNavegacion['url_base'].apply(lambda url: str(url).split('?')[0])
print(dfNavegacion['url_base'].head(50))

In [ ]:
# Obtener el modelo de coche y agregarlo a una nueva columna
def obtenerModelo(url):
  posicion = url.rfind('/')    # busca la ultima barra
  modeloObtenido = url[posicion+1 :]
  if modeloObtenido == '' or modeloObtenido == 'renting':
    modeloObtenido = 'home'
  return modeloObtenido

dfNavegacion.insert(loc=7, column='modelo', value=dfNavegacion['url_base'].apply(obtenerModelo))
print(dfNavegacion['modelo'].head(50))

Crear un orden de usuarios: <br/>
Como tenemos usuarios repetidos me daria problema al hacer el merge con el dataframe de conversiones <br/>
Voy a crear una columna llamada orden que almacena el orden de los id_user <br/>
Una vez que tenga ese orden, filtro del dfNavegacion para quedarme solo con el primero (orden=0)

In [17]:
# ordenar por id_user y ts
dfNavegacion = dfNavegacion.sort_values(['id_user','ts'])
dfNavegacion.insert(loc=1, column='orden', value=dfNavegacion.groupby('id_user')['ts'].cumcount())

# Filtrar por orden=0
dfNavegacion = dfNavegacion[dfNavegacion['orden']==0]

Cargar el fichero de conversiones

In [18]:
uploaded = files.upload()

Saving conversiones.csv to conversiones.csv


Crear el dataframe de conversiones

In [ ]:
dfConversiones = pd.read_csv('conversiones.csv', sep=";")
print(dfConversiones.head(10))

Vamos a hacer el merge con modelo=outer para no perder ningun dato

In [20]:
dfFinal = dfNavegacion.merge(dfConversiones, how='outer', on=['id_user'])

Para saber si hay conversion o no. <br/>
Crear una columna llamada conversiones con el valor 1 si es conversion, 0 si no lo es

In [ ]:
def esConversion(dato):
  lista = ['CALL','FORM']
  if dato in lista:
    return 1
  else:
    return 0

dfFinal.insert(loc=1, column="conversiones", value=dfFinal['lead_type'].apply(esConversion))
print(dfFinal)

<h1> ********   Preguntas a responder ******** </h1>
<h2>Pregunta 1: Cuantas visitas recibe en el dia el cliente?</h2>

In [22]:
# Como la columna ts contiene fecha y hora, creamos una nueva columna dia solo con la fecha aplicando split(' ')
dfFinal.insert(loc=1, column="dia", value=dfFinal['ts'].apply(lambda ts: str(ts).split(' ')[0]))
print(dfFinal.groupby('dia')['dia'].count())

dia
2021-09-06    5640
2021-09-07       4
nan             13
Name: dia, dtype: int64


<h3>Cuantas de ellas convierten y cuantas no (en %)</h3>

In [23]:
# Filtrar por 2021-09-06
visitas_dia_6_septiembre = dfFinal[dfFinal['dia'] == '2021-09-06']
num_visitas_dia = visitas_dia_6_septiembre['dia'].count()

# cuento las visitas con conversion
visitas_convierten = visitas_dia_6_septiembre[visitas_dia_6_septiembre['conversiones']==1]['ts'].count()

# cuento las visitas sin conversion
visitas_no_convierten = visitas_dia_6_septiembre[visitas_dia_6_septiembre['conversiones']==0]['ts'].count()

# calculo los porcentajes
porcentaje_visitas_convierten = round( (visitas_convierten / num_visitas_dia) * 100, 2)
porcentaje_visitas_no_convierten = round( (visitas_no_convierten / num_visitas_dia) * 100, 2)
print("Convierten:", porcentaje_visitas_convierten)
print("No convierten:", porcentaje_visitas_no_convierten)


Convierten: 0.27
No convierten: 99.73


<h2>Pregunta 2: Por tipo de conversion (CALL o FORM), cuantas hay de cada una</h2>

In [24]:
print(dfFinal.groupby('lead_type')['lead_type'].count())

lead_type
CALL    19
FORM     9
Name: lead_type, dtype: int64


<h2>Pregunta 3: Porcentaje de usuarios recurrentes sobre el total de usuarios</h2>

In [25]:
# Contar cuantos usuarios tenemos
total_usuarios = dfFinal['id_user'].count()

# Contar los usuarios recurrentes
usuarios_recurrentes = dfFinal[dfFinal.user_recurrent & dfFinal.user_recurrent.notnull()]['ts'].count()

# calculamos el porcentaje
porcentaje = round((usuarios_recurrentes / total_usuarios) * 100 , 2)
print("Usuarios recurrentes: ", porcentaje)

Usuarios recurrentes:  18.7


<h2>Pregunta 4: cual es el coche mas visitado</h2>

In [ ]:
#print(dfFinal.groupby('modelo')['modelo'].count())
print(dfFinal['modelo'].value_counts())  # muestra los datos ordenados descendente

<h3>Es el que mas convierte?</h3>

In [30]:
# Voy a filtrar el dataset para tener solo los registros que han convertido y 
# luego los agrupo por modelo
print(dfFinal[dfFinal['conversiones'] == 1].groupby('modelo')['conversiones'].count())


modelo
cea-electrico    1
clin200          2
clin400          3
dep30            1
home             5
ixs-electrico    1
tria             2
Name: conversiones, dtype: int64


<h1>Exportar el dfFinal a un Excel para elaborar informes, graficos, histogramas, ...etc</h2>

In [31]:
dfFinal.to_excel("dataframe_final.xlsx", index=False)